<a href="https://colab.research.google.com/github/CarOmodeo/practicos_pnl/blob/main/Desafio_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio 2 - Vectores con Gensim

## Alumno: Carolina Pérez Omodeo


- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

## Datos

Se utilizaran los libros de "Orgullo y Prejuicio" de Jane Austen para desarrollar el desafio.

In [3]:
from google.colab import files
uploaded = files.upload()


Saving orgullo_y_prejuicio.txt to orgullo_y_prejuicio.txt


In [4]:
df = pd.read_csv('orgullo_y_prejuicio.txt', sep='/n', header=None, engine = 'python')
df.head()

,0
0,Orgullo y Prejuicio
1,Jane Austen
2,Capítulo I
3,Es una verdad mundialmente reconocida que un h...
4,"Sin embargo, poco se sabe de los sentimientos ..."


In [5]:
print("Total de documentos: ", df.shape[0])

Total de documentos:  2196


## Etapa de Procesamiento

In [6]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

# Paso del texto a listas

sentence_tokens = []

for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [8]:
# Listas formadas
sentence_tokens[:5]

[['orgullo', 'y', 'prejuicio'],
 ['jane', 'austen'],
 ['capítulo', 'i'],
 ['es',
  'una',
  'verdad',
  'mundialmente',
  'reconocida',
  'que',
  'un',
  'hombre',
  'soltero',
  'poseedor',
  'de',
  'una',
  'gran',
  'fortuna',
  'necesita',
  'una',
  'esposa'],
 ['sin',
  'embargo',
  'poco',
  'se',
  'sabe',
  'de',
  'los',
  'sentimientos',
  'u',
  'opiniones',
  'de',
  'un',
  'hombre',
  'de',
  'tales',
  'condiciones',
  'cuando',
  'entra',
  'a',
  'formar',
  'parte',
  'de',
  'un',
  'vecindario',
  'esta',
  'verdad',
  'está',
  'tan',
  'arraigada',
  'en',
  'las',
  'mentes',
  'de',
  'algunas',
  'de',
  'las',
  'familias',
  'que',
  'lo',
  'rodean',
  'que',
  'algunas',
  'le',
  'consideran',
  'de',
  'su',
  'legítima',
  'propiedad',
  'y',
  'otras',
  'de',
  'la',
  'de',
  'sus',
  'hijas']]

## Etapa de creación de vectores (word2vec)

In [9]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [11]:
# Se crea el modelo generador de vectores, con skipgram

w2v_model = Word2Vec(min_count=10,      # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=10,          # cant de palabras antes y desp de la predicha
                     vector_size=200,   # dimensionalidad de los vectores
                     negative=3,        # cantidad de negative samples... 0 es no se usa
                     workers=2,        # si tienen más cores pueden cambiar este valor
                     sg=1)              # modelo 0:CBOW  1:skipgram

In [12]:
# Vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [13]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 2196


In [14]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 1186


## Etapa de entrenamiento de embeddings

In [15]:
# Entrenamos el modelo generador de vectores
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=1001,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 675244.4375
Loss after epoch 1: 634549.9375
Loss after epoch 2: 635147.75
Loss after epoch 3: 633993.625
Loss after epoch 4: 634684.75
Loss after epoch 5: 632958.75
Loss after epoch 6: 614878.75
Loss after epoch 7: 584730.5
Loss after epoch 8: 593767.5
Loss after epoch 9: 591055.0
Loss after epoch 10: 590627.5
Loss after epoch 11: 574995.0
Loss after epoch 12: 572076.0
Loss after epoch 13: 518686.5
Loss after epoch 14: 467907.0
Loss after epoch 15: 472050.0
Loss after epoch 16: 469433.0
Loss after epoch 17: 437469.0
Loss after epoch 18: 473432.0
Loss after epoch 19: 475287.0
Loss after epoch 20: 475521.0
Loss after epoch 21: 470192.0
Loss after epoch 22: 474858.0
Loss after epoch 23: 440421.0
Loss after epoch 24: 464873.0
Loss after epoch 25: 435424.0
Loss after epoch 26: 469449.0
Loss after epoch 27: 467680.0
Loss after epoch 28: 476523.0
Loss after epoch 29: 476554.0
Loss after epoch 30: 476778.0
Loss after epoch 31: 454961.0
Loss after epoch 32: 353156.0
Loss aft

(59338010, 115269154)

## Pruebas

### Pruebas de interés

In [31]:
# Palabras que MÁS se relacionan con "matrimonio"
w2v_model.wv.most_similar(positive=["matrimonio"], topn=10)

[('felicidad', 0.33844903111457825),
 ('proposición', 0.28578025102615356),
 ('fortuna', 0.2617620527744293),
 ('recibió', 0.25271058082580566),
 ('único', 0.2516445219516754),
 ('casada', 0.24860697984695435),
 ('seguro', 0.2477155327796936),
 ('juventud', 0.2438347488641739),
 ('disgusto', 0.24175982177257538),
 ('sabía', 0.23984849452972412)]

Las palabras más relacionadas con "matrimonio" incluyen términos como "felicidad", "proposición" y "fortuna", lo que sugiere que el concepto de matrimonio en la novela está vinculado con la idea de bienestar emocional y estabilidad financiera (en el caso de familias que tenian solo hijas mujeres). También aparecen palabras como "casada" y "juventud", lo que podría reflejar la importancia de la juventud y las expectativas sociales sobre el matrimonio, junto con la referencia al "disgusto", que podría señalar las tensiones o desafíos asociados con este tema.

In [34]:
# Palabras que MENOS se relacionan con la palabra "matrimonio"
w2v_model.wv.most_similar(negative=["matrimonio"], topn=10)

[('bennet—', 0.13732989132404327),
 ('papá', 0.10017896443605423),
 ('buena', 0.09543897956609726),
 ('estos', 0.08292658627033234),
 ('triste', 0.0802323967218399),
 ('temer', 0.07793125510215759),
 ('camino', 0.07552922517061234),
 ('decidió', 0.07211490720510483),
 ('realmente', 0.07154206186532974),
 ('dime', 0.07100652158260345)]

Las palabras menos relacionadas con "matrimonio" incluyen términos como "bennet—", "papá" y "buena", que son más generales y no están directamente vinculadas a la institución del matrimonio. También aparecen palabras como "triste" y "temer", que reflejan emociones negativas y temores que no suelen asociarse con las ideas positivas o las expectativas sociales que se tienen sobre el matrimonio en la novela.

In [33]:
# Palabras que MÁS se relacionan con "orgullo"
w2v_model.wv.most_similar(positive=["orgullo"], topn=10)

[('vanidad', 0.33086904883384705),
 ('afecto', 0.28651466965675354),
 ('hijo', 0.2750069499015808),
 ('cualidades', 0.26026734709739685),
 ('generoso', 0.24782130122184753),
 ('resentimiento', 0.23501630127429962),
 ('alto', 0.23411493003368378),
 ('caso', 0.23210176825523376),
 ('él', 0.23179791867733002),
 ('dice', 0.22903448343276978)]

Las palabras relacionadas con "orgullo" como "vanidad", "afecto" y "hijo" indican que el orgullo en la novela está vinculado tanto a la autoimagen como a las relaciones familiares. La presencia de términos como "generoso" y "resentimiento" refleja el conflicto entre las cualidades personales y las emociones negativas que el orgullo puede generar en las interacciones, mientras que "alto" y "caso" pueden sugerir distinciones sociales y situaciones donde el orgullo juega un papel importante en los juicios de los personajes.

In [35]:
# Palabras que MENOS se relacionan con "orgullo"
w2v_model.wv.most_similar(negative=["orgullo"], topn=10)

[('sobrina', 0.11618072539567947),
 ('mano', 0.10838272422552109),
 ('respuesta', 0.1027355045080185),
 ('apenas', 0.09173315018415451),
 ('absolutamente', 0.08625943958759308),
 ('decidida', 0.08038736134767532),
 ('tuvo', 0.0785980373620987),
 ('seguida', 0.07518467307090759),
 ('dirección', 0.07401349395513535),
 ('pudiera', 0.07262668758630753)]

Las palabras menos relacionadas con "orgullo" incluyen términos como "sobrina", "respuesta" y "apenas", que indican interacciones más neutras o cotidianas. Otras como "decidida" y "tuvo" sugieren decisiones o acciones que no están impulsadas por el orgullo, sino por otros factores como la determinación o las circunstancias.

In [32]:
# Palabras que MÁS se relacionan con "honor"
w2v_model.wv.most_similar(positive=["honor"], topn=10)

[('bailar', 0.24975107610225677),
 ('obligado', 0.24886076152324677),
 ('menor', 0.24734173715114594),
 ('haciendo', 0.2431022822856903),
 ('convencido', 0.2407100647687912),
 ('inconveniente', 0.23863445222377777),
 ('leer', 0.23241634666919708),
 ('estima', 0.23049935698509216),
 ('mano', 0.21645274758338928),
 ('hiciese', 0.21461975574493408)]

Las palabras más relacionadas con el término "honor" se asocian con palabras como "bailar", "obligado" y "menor", lo que indica que el honor en la novela puede estar relacionado con normas sociales y expectativas de comportamiento en eventos formales, como por ejemplo los bailes. También se encuentran términos como "convencido" y "inconveniente", lo que sugiere que el honor está conectado con las decisiones y la presión social de la época, mientras que "estima" y "mano" reflejan la importancia del respeto y el compromiso en las relaciones.

In [36]:
# Palabras que MENOS se relacionan con "honor"
w2v_model.wv.most_similar(negative=["honor"], topn=10)

[('menudo', 0.12300035357475281),
 ('juntos', 0.09283412992954254),
 ('dentro', 0.09144085645675659),
 ('encontrado', 0.08942659199237823),
 ('venir', 0.08901205658912659),
 ('te', 0.08323408663272858),
 ('desagradable', 0.08149594068527222),
 ('casos', 0.0800391137599945),
 ('salió', 0.07840721309185028),
 ('desde', 0.07780750095844269)]

Las palabras menos relacionadas con "honor" incluyen términos como "menudo", "juntos" y "dentro", que no tienen una conexión directa con el concepto de honor en el contexto de la novela. Palabras como "desagradable" y "salió" reflejan situaciones más triviales o negativas que no se asocian con las ideas de respeto y dignidad que el honor implica en la trama.

### Prueba de analogía

In [44]:
result = w2v_model.wv.most_similar(positive=['elizabeth', 'bingley'], negative=['darcy'], topn=1)
print(result)

[('jane', 0.3704480528831482)]


El modelo ha identificado que la palabra más relacionada con la analogía "Elizabeth + Bingley - Darcy", es "Jane" (0.37 de similitud). Esto tiene sentido dentro del contexto de la novela, ya que Jane Bennet, la hermana de Elizabeth, tiene una relación amorosa con Mr. Bingley. La conexión entre Jane y Bingley es muy similar a la relación entre Elizabeth y Darcy. Es un reflejo de cómo el modelo captura la idea de dos personajes que están en relaciones amorosas paralelas.

In [50]:
result = w2v_model.wv.most_similar(positive=['bennet', 'elizabeth'], negative=['lydia'], topn=1)
print(result)

[('señora', 0.45436811447143555)]


La analogía "Bennet + Elizabeth - Lydia", "señora" (0.45 de similitud), podría reflejar la relación general de la madre con sus hijas, en este caso con Lydia, quien representa una hija más rebelde y menos contenida que Elizabeth. Esta respuesta podría interpretarse como un reflejo de la autoridad de la madre sobre sus hijas, especialmente en una sociedad donde la figura materna tenía una gran influencia en las decisiones familiares, como el matrimonio.

## Visualización de agrupaciones de vectores

En los gráficos se observará las relaciones entre las palabras. Se utilizá IncrementalPCA para reducir la dimencionalidad de los vectores de palabras para poder observar una mejor cercanía entre ellos.

In [52]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions_TSNE(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

def reduce_dimensions_IPCA(model, num_dimensions=2, batch_size=100):
  vectors = np.asarray(model.wv.vectors)
  labels = np.asarray(model.wv.index_to_key)

  # Inicializamos IncrementalPCA
  ipca = IncrementalPCA(n_components=num_dimensions, batch_size=batch_size)

  # Ajustamos y transformamos los vectores
  vectors = ipca.fit_transform(vectors)

  return vectors, labels

In [59]:
# Plot 2D de embeddings con Incremental_PCA
import plotly.graph_objects as go
import plotly.express as px

# Parámetros
MAX_WORDS = 100
vecs, labels = reduce_dimensions_IPCA(w2v_model)

# Plot
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=vecs[:MAX_WORDS, 0],
        y=vecs[:MAX_WORDS, 1],
        mode='markers+text',
        text=labels[:MAX_WORDS],
        textposition="top center",
        textfont=dict(size=9),
        marker=dict(size=6, color='blue', opacity=0.7),
        hoverinfo='text',
    )
)

# Formato del gráfico
fig.update_layout(
    title="Proyección 2D - Embeddings con Incremental PCA",
    xaxis_title="Componente 1",
    yaxis_title="Componente 2",
    template="plotly_white",
    showlegend=False,
    autosize=True,
    margin=dict(l=40, r=40, t=40, b=40),  # Márgenes ajustados
)

# Mostrar el gráfico
fig.show()

Del siguiente grafico se puede concluir que:


  *   Los personajes como "wickham", "darcy", "jane" y "bingley" están ubicados en diferentes zonas del gráfico, lo que podría reflejar las diferentes interacciones que tienen entre sí a lo largo de la novela.
  *   "Darcy" y "bingley", que tienen relaciones importantes con "jane" y "elizabeth", están relativamente cerca unas de otras, pero no forman un grupo cohesivo. Indicando que, en los embeddings de palabras, sus contextos no son necesariamente coincidentes en todos los casos, aunque en la trama estén vinculadas a través de relaciones complejas.
  * Hay palabras como "señora", "señorita" y "familia", pero su relación no está tan próxima como podría esperarse. Esto sugiere que, aunque estas palabras se utilizan en situaciones de relaciones de clase social y matrimonio, los embeddings las tratan como términos más contextuales, dispersos y vinculados a un conjunto más amplio de temas.
  * "Elizabeth" y "bennet" están en posiciones separadas, lo que puede reflejar que, aunque están estrechamente relacionadas en el contexto de la novela, en términos de sus embeddings de palabras no son las más cercanas entre sí.


In [57]:
# Gráficos 3D de embeddings con Incremental_PCA

vecs, labels = reduce_dimensions_IPCA(w2v_model,3)

# Plot
fig = go.Figure()

fig.add_trace(
    go.Scatter3d(
        x=vecs[:MAX_WORDS, 0],
        y=vecs[:MAX_WORDS, 1],
        z=vecs[:MAX_WORDS, 2],
        mode='markers+text',
        marker=dict(
            size=4,
            color='blue',
            opacity=0.7
        ),
        text=labels[:MAX_WORDS],
        textposition='top center',
        textfont=dict(size=9),
        hoverinfo='text',
    )
)

# Personalizar el diseño del gráfico
fig.update_layout(
    title="Proyección 3D - Embeddings con Incremental PCA",
    scene=dict(
        xaxis_title="Componente 1",
        yaxis_title="Componente 2",
        zaxis_title="Componente 3",
    ),
    template="plotly_white",
    showlegend=False,  # Ocultar leyenda
    margin=dict(l=0, r=0, t=40, b=0),  # Márgenes ajustados
)

# Mostrar el gráfico
fig.show()

Al agregar una dimensión (componente) más, se pueden observar mejor las cercanías entre palabras.

# Colclusiones

El modelo de embeddings ha logrado reflejar de manera precisa el contexto semántico de Orgullo y prejuicio, evidenciando cómo los términos y conceptos clave están conectados entre sí. Palabras como "señor", "familia", "hermana", "Bennet", "Elizabeth", "Jane" y "señorita" destacan las dinámicas familiares y sociales que dominan la trama, enfocándose en temas como el matrimonio y las relaciones de clase en la sociedad de la época.

Las pruebas de analogías han desvelado asociaciones semánticas reveladoras. Por ejemplo, la relación entre "Elizabeth + Bingley - Darcy" que da como resultado "Jane", y la analogía "Bennet + Elizabeth - Lydia" que da como "señora", reflejan las complejas interacciones entre los personajes y las tensiones dentro de la familia Bennet. Estos resultados indican que el modelo captura las relaciones emocionales y sociales fundamentales que mueven la narrativa de la novela.

La reducción de dimensionalidad utilizando Incremental PCA ha permitido visualizar cómo se agrupan las palabras en función de su significado y contexto. Palabras relacionadas con los personajes, sus acciones y el desarrollo de la trama, como "dijo", "estaba", "tenía" y "era", se agrupan de manera coherente, mientras que términos como "señor" y "señorita" se dispersan, lo que refleja las distinciones de clase y estatus social que son cruciales en el texto. Esta visualización refuerza cómo las relaciones interpersonales y los valores sociales se entrelazan de manera significativa a lo largo de la novela.